In [1]:
import numpy as np 
import os 
import matplotlib.pyplot as plt 

In [2]:
import json
# Example of how to load the saved data for training
def load_data(data_path='processed_3_data_normalized'):
    # Load the keypoints and labels
    X = np.load(os.path.join(data_path, 'X.npy'))
    y = np.load(os.path.join(data_path, 'y.npy'))
    
    # Load the label mapping
    with open(os.path.join(data_path, 'action_labels.json'), 'r') as f:
        action_to_label = json.load(f)
    
    # Create reverse mapping (numerical label to action name)
    label_to_action = {int(v): k for k, v in action_to_label.items()}
    
    return X, y, action_to_label, label_to_action

# Load the data
X, y, action_to_label, label_to_action = load_data()
print(f"Loaded data shape - X: {X.shape}, y: {y.shape}")
print(f"\nNumber of sequences for each action:")
for label in np.unique(y):
    action = label_to_action[label]  # Now using integer label directly
    count = np.sum(y == label)
    print(f"{action}: {count} sequences")

Loaded data shape - X: (1332, 30, 258), y: (1332,)

Number of sequences for each action:
alive: 24 sequences
bad: 21 sequences
Beautiful: 24 sequences
big large: 21 sequences
Blind: 24 sequences
cheap: 24 sequences
clean: 24 sequences
cold: 20 sequences
cool: 21 sequences
curved: 24 sequences
dead: 24 sequences
Deaf: 24 sequences
deep: 24 sequences
dirty: 24 sequences
dry: 21 sequences
expensive: 24 sequences
famous: 24 sequences
fast: 21 sequences
female: 24 sequences
flat: 24 sequences
good: 21 sequences
happy: 21 sequences
hard: 24 sequences
healthy: 21 sequences
heavy: 24 sequences
high: 24 sequences
hot: 21 sequences
light: 24 sequences
long: 21 sequences
loose: 24 sequences
loud: 21 sequences
low: 24 sequences
male: 24 sequences
Mean: 24 sequences
narrow: 21 sequences
new: 21 sequences
Nice: 20 sequences
old: 21 sequences
poor: 24 sequences
quiet: 21 sequences
rich: 24 sequences
sad: 24 sequences
shallow: 24 sequences
short: 21 sequences
sick: 21 sequences
slow: 21 sequences
smal

In [3]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout, Bidirectional,GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train,x_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=2)

In [7]:
X_train.shape
print(y_train.shape)

(1065,)


In [8]:
labels = y.copy()  # integer labels (0..num_classes-1)
num_classes = len(np.unique(labels))
y = to_categorical(labels, num_classes=num_classes).astype(int)
print('num_classes =', num_classes, 'labels shape =', labels.shape, 'y(one-hot) shape =', y.shape)

num_classes = 59 labels shape = (1332,) y(one-hot) shape = (1332, 59)


In [9]:
model = Sequential([
    LSTM(128, return_sequences=True, activation='selu', input_shape=(30, 258)),
    BatchNormalization(),
    Dropout(0.3),

    LSTM(64, return_sequences=True, activation='selu'),
    BatchNormalization(),
    Dropout(0.25),

    LSTM(64, return_sequences=False, activation='selu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='elu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(32, activation='elu'),
    BatchNormalization(),
    Dropout(0.2),

    Dense(59, activation='softmax')
])

# Compile the model
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Build the model to enable summary
# model.build(input_shape=(None, 30, 258))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           198144    
                                                                 
 batch_normalization (BatchN  (None, 30, 128)          512       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 30, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 30, 64)            49408     
                                                                 
 batch_normalization_1 (Batc  (None, 30, 64)           256       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 30, 64)            0

In [10]:
import datetime
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [11]:
# Callbacks
early = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Training hyperparameters
epochs = 500
batch_size = 16

history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[tensorboard_callback, early, reduce_lr],
    shuffle=True
)

Epoch 1/500


ValueError: in user code:

    File "c:\Users\lokes\miniconda3\envs\isl_env\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\lokes\miniconda3\envs\isl_env\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\lokes\miniconda3\envs\isl_env\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\lokes\miniconda3\envs\isl_env\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\lokes\miniconda3\envs\isl_env\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "c:\Users\lokes\miniconda3\envs\isl_env\lib\site-packages\keras\engine\compile_utils.py", line 2

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_accuracy*100:.2f}%")

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['categorical_accuracy'], label='Training Accuracy')
plt.plot(history.history['val_categorical_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

